In [2]:
import json 
import os 

In [ ]:
train_data = "raw/python100k_train.json"
with open(train_data, 'r', encoding='utf-8') as f: 
    print(f)
    all_data = []
    for line in f:
        print(line)
        data = json.loads(line)
        all_data.append(data)

len(all_data)